In [2]:
import numpy as np
from my_nmt_utils import *
from nmt_utils import *

from keras.layers import Bidirectional, LSTM, Dense, Activation, RepeatVector, Lambda, Concatenate, Permute, Dot, Input, Multiply
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K

Using TensorFlow backend.


# Select Sets:

In [3]:
french_vocab = get_vocab('french_set.txt')
french_vocab = {word:i for i, word in enumerate(french_vocab)}
inv_french_vocab = {i:word for i, word in enumerate(french_vocab)}

english_vocab = get_vocab('english_set.txt')
english_vocab = {word:i for i, word in enumerate(english_vocab)}

total words in vacab: 382
total words in vacab: 323


In [4]:
french_data = np.genfromtxt('french_set.txt', dtype = 'str', delimiter = ',', encoding = 'utf8')
english_data = np.genfromtxt('english_set.txt', dtype = 'str', delimiter = ',', encoding = 'utf8')

max_len_fr = get_max_length(french_data)
max_len_en = get_max_length(english_data)

# Visualization 

In [5]:
Ten = max_len_en
Tfr = max_len_fr
fr, fr_oh = convert_string_data_to_onehot(french_data, french_vocab, Tfr)
en, en_oh = convert_string_data_to_onehot(english_data, english_vocab, Ten)
print("fr.shape:", fr.shape)
print("en.shape:", en.shape)
print("fr_oh.shape:", fr_oh.shape)
print("en_oh.shape:", en_oh.shape)

fr.shape: (200, 12)
en.shape: (200, 11)
fr_oh.shape: (200, 12, 382)
en_oh.shape: (200, 11, 323)


# Model

In [6]:
repeator = RepeatVector(Ten)
concatenator = Concatenate(axis = -1)
densor = Dense(1, activation = 'relu')
activator = Activation('softmax')
dotor = Dot(axes = 1)

In [7]:
def one_step_attension(a, s_prev):
    # a = hidden state of Bi-LSTM of shape (m, Tx, 2*n_a)
    # s_prev = previous hidden state of post attension LSTM layer (creating as post_att.. = LSTM) of shape (m, n_s)
    
    s_prev = repeator(s_prev)           # so that its shape becomes (m, Tx, n_s) to concatenate with a
    
    concat = concatenator([a, s_prev])
    
    energy = densor(concat)
    
    alpha = activator(energy)
    
    context = dotor([alpha, a])
    
    return context

In [8]:
n_a = 64
n_s = 2*n_a
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(french_vocab), activation=softmax)

In [9]:
def model(Tx, Ty, n_a, n_s, english_vocab_size, french_vocab_size, max_len_fr):
    
    X = Input(shape=(Tx, english_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences = True))(X)
    
    counter = 0
    stop = False
    while not stop:
        context = one_step_attension(a, s)
        
        s, _, c = post_activation_LSTM_cell(context, initial_state = [s, c])       # s= hidden state, c= cell state
        
        out = output_layer(s)
        
        outputs.append(out)
        
        counter += 1
        
        if counter == max_len_fr:
            stop = True
        
    model = Model(inputs = [X, s0, c0], outputs = outputs)
    
    return model

In [10]:
nm_model = model(Ten, Tfr, n_a, n_s, len(english_vocab), len(french_vocab), max_len_fr)

In [11]:
out = nm_model.compile(optimizer=Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01),
                    metrics=['accuracy'],
                    loss='categorical_crossentropy')

In [12]:
m = english_data.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(fr_oh.swapaxes(0,1))

In [19]:
nm_model.fit([en_oh, s0, c0], outputs, epochs = 1, batch_size= 16)

Epoch 1/1
200/200 [==============================] - ETA: 2s - loss: 15.1838 - dense_2_loss_1: 2.7690 - dense_2_loss_2: 3.2089 - dense_2_loss_3: 3.5787 - dense_2_loss_4: 2.2973 - dense_2_loss_5: 1.0891 - dense_2_loss_6: 1.0867 - dense_2_loss_7: 0.6397 - dense_2_loss_8: 0.2224 - dense_2_loss_9: 0.2652 - dense_2_loss_10: 0.0143 - dense_2_loss_11: 0.0071 - dense_2_loss_12: 0.0053 - dense_2_acc_1: 0.2500 - dense_2_acc_2: 0.3125 - dense_2_acc_3: 0.3125 - dense_2_acc_4: 0.4375 - dense_2_acc_5: 0.6875 - dense_2_acc_6: 0.7500 - dense_2_acc_7: 0.8750 - dense_2_acc_8: 0.9375 - dense_2_acc_9: 0.9375 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.00 - ETA: 2s - loss: 14.9476 - dense_2_loss_1: 2.7947 - dense_2_loss_2: 3.3270 - dense_2_loss_3: 3.3466 - dense_2_loss_4: 2.5023 - dense_2_loss_5: 1.1072 - dense_2_loss_6: 1.1424 - dense_2_loss_7: 0.3977 - dense_2_loss_8: 0.1653 - dense_2_loss_9: 0.1415 - dense_2_loss_10: 0.0114 - dense_2_loss_11: 0.0064 - dense_2_loss_12: 0.0051 - 

# Test Your Own Sentences

In [20]:
EXAMPLES = ['i love you', 'i hate you', 'you are my love', 'steve is genius', 'ha ha ha wow']
for examples in EXAMPLES:
    source = convert_example_to_indices(examples, english_vocab, Ten)
    source = convert_to_one_hot(source, C= len(english_vocab)).reshape(1, Ten, len(english_vocab))
    predict = nm_model.predict([source, s0, c0])
    predict = np.argmax(predict, axis = -1)
    output  = [inv_french_vocab[int(i)] for i in predict]
    
    print("source:", examples)
    print("outputs:", ' '.join(output))

source: i love you
outputs: je t'aime . . . . . . . . . .
source: i hate you
outputs: je veux . . . . . . . . . .
source: you are my love
outputs: tu es mon . . . . . . . . .
source: steve is genius
outputs: elle est est . . . . . . . . .
source: ha ha ha wow
outputs: ha ha ha . . . . . . . . .
